# Generating motions with feature and pseudoinverse

Let $y = \phi(q)$ be a feature (a mapping from joint to task space) and $J(q) = \frac{\partial\phi}{\partial q}$ be a Jacobian. From this mapping, we can relate the joint velocity to the velocity of the feature value, $$\dot{y} = J(q)\dot{q}.$$

Suppose that we want the feature value to have a particular dynamics $\dot{y}_d = f(y_d)$, i.e. a desired feature motion. So as to achieve such a feature dynamics, how should the joint move? Such a question might be formalized as an optimization problem: $$\dot{q}^*=\text{arg}\min_{\dot{\tilde{q}}}||\dot{y}_d-J(q)\dot{\tilde{q}}||^2.$$ One straightforward answer is from the psuedoinverse:
$$\dot{q}^* = J(q)^+\dot{y}_d.$$
1. When $\dim(y) < \dim(q)$ and J has a full row rank, there exist the infinite number of solutions $\dot{\tilde{q}}$ that satisfy $\dot{y}_d = J(q)\dot{\tilde{q}}$ and the psuedoinverse formula picks the minimum joint velocity (in the Euclidean sense) among those.
2. When $\dim(y) > \dim(q)$, no solution exists and the psuedoinverse provides the least-square solution.
3. When $\dim(y) = \dim(q)$ and J has a full rank, the psuedoinverse reduces to the ordinary inverse.

More generally, a weighted Euclidean metric as well as a regularization term can be used to get better numerical stability and deal with edge cases (see the lecture slides!).

In [1]:
import sys
sys.path.append('../../build')
import libry as ry
import numpy as np
import time

**ry-c++-log** /home/heng/git/robotics-course/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback



In [2]:
#-- MODEL WORLD configuration, this is the data structure on which you represent
# what you know about the world and compute things (controls, contacts, etc)
C = ry.Config()
C.addFile("../../scenarios/pandasTable.g")
D = C.view()
q0 = C.getJointState()

In [4]:
obj = C.addFrame("object")
# set frame parameters, associate a shape to the frame, 
pos_obj = np.array([1.,0,1.5])
pos_obj0 = pos_obj.copy()
obj.setPosition(pos_obj)
obj.setQuaternion([1,0,0,0])
obj.setShape(ry.ST.sphere, [.02])
obj.setColor([1,0,0])

## Tracking an object

Suppose the feature $\phi$ is a mapping to the position difference between the gripper and the object (case 1 above). If this feature space is endowed with a stable dynamics, e.g., $\dot{y}=-y$, corresponding joint motions will let the gripper position to approach to the object (i.e., the position difference converges to 0). The following code exactly does that.

In [7]:
tau = .01
obj.setPosition(pos_obj)
C.setJointState(q0)
q = C.getJointState()
for t in range(1000):
    time.sleep(0.01)
    pos_obj[0] = 1 - np.sin(0.01*t) #move the obj for fun!
    obj.setPosition(pos_obj)

    [y,J] = C.evalFeature(ry.FS.positionRel, ["R_gripperCenter", "object"])

    vel_ee = -y
    #vel = np.linalg.pinv(J) @ vel_ee; #less stable!
    vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ vel_ee; #psuedoinverse with a regularization term
    #vel = np.linalg.inv(J.T@J + 1e-2*np.eye(q.shape[0])) @ J.T @ vel_ee;

    q += tau*vel
    C.setJointState(q) #set your robot model to match the real q

In [8]:
tau = .01
C.setJointState(q0)
obj.setPosition(pos_obj0)
q = C.getJointState()
for t in range(500):
    time.sleep(0.01)
    
    #evaluate a first feature
    [y1,J1] = C.evalFeature(ry.FS.positionRel, ["R_gripperCenter", "object"])
    
    #you can multiply y1 and J1 here with some number, to increase the importance of the first feature
    #evaluate a second feature
    [y2,J2] = C.evalFeature(ry.FS.scalarProductXY, ["R_gripper","world"])

    #you can multiply y2 and J2 here with some number, to increase the importance of the second feature
    #stack all tasks
    vel_ee = np.block([-y1, 1.-y2])
    J = np.block([[J1],[J2]])
    
#     vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ vel_ee;
    
    vel_ee = np.block([-y1, 1.-y2, np.zeros(q.shape[0])]) #Third "qItself" feature is for regularization (small joint velocity)
    J = np.block([[J1],[J2],[1e-2*np.eye(q.shape[0])]])
    vel =  np.linalg.pinv(J) @ vel_ee;
    
    ## which is actaully equivalent to this!
#     vel =  np.linalg.inv(J.T@J + 1e-2*np.eye(q.shape[0])) @ J.T @ vel_ee;


    q += tau*vel
    C.setJointState(q) 

## Circling motion with coordinate transformation 

The circling motion can be easily made in the polar cordinate, e.g., $$\dot{r} = 1-r,~\dot{\theta}=1.$$
So, all we need to do is:
1. transform the feature value (again only $(y_1,y_3)$ components, with $\dot{y}_2=-y_2$) into polar coordinate, $$r=\sqrt{y_1^2+y_3^2},~\theta=\arctan2(y_1,y_3).$$
2. compute the desired velocity in polar coordinate, $$\dot{r} = 1-r,~\dot{\theta}=1.$$
3. transform it back to the original feature space, $$\dot{y}_1 = \dot{r}\cos\theta-r\dot{\theta}\sin\theta,\\
\dot{y}_3 = \dot{r}\sin\theta+r\dot{\theta}\cos\theta.$$
(derived from $y_1=r\cos\theta, y_3=r\sin\theta$)
4. take the psuedoinverse to get the joint velocity.

In [14]:
tau = .01
C.setJointState(q0)
q = C.getJointState()
for t in range(500):
    time.sleep(0.01)
    
    [y,J] = C.evalFeature(ry.FS.positionRel, ["R_gripperCenter", "object"])
    #pos_obj[0] = 1 - np.sin(0.01*t) #move the obj for fun!
    obj.setPosition(pos_obj)

    # 1. transform to polar coordinate
    r = np.sqrt(y[0]**2 + y[2]**2)
    theta = np.arctan2(y[2],y[0])

    # 2. compute the velocity there (you can change the params of dynamics)
    r_dot = 0.2-r
    theta_dot = -5.

    # 3. transform back to feature space (gripper velocity)
    y1_dot = r_dot*np.cos(theta)-r*theta_dot*np.sin(theta)
    y3_dot = r_dot*np.sin(theta)+r*theta_dot*np.cos(theta)
    
    y2_dot = -y[1]

    # 4. take the psuedoinverse to get the joint velocity.
    vel_ee = np.array([y1_dot, y2_dot, y3_dot])
    vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ vel_ee;
#     vel =  np.linalg.inv(J.T@J + 1e-2*np.eye(q.shape[0])) @ J.T @ vel_ee;
    
    q += tau*vel
    C.setJointState(q) 

In [7]:
S=0
C=0
RealWorld=0